In [1]:
import numpy as np
from skfem import (
    MeshTri, Basis, FacetBasis, ElementTriP2, ElementTriP1, ElementVector,
    asm, bmat, condense, solve, BilinearForm, LinearForm, Mesh, ElementTetP2, ElementTetP1
)
from skfem.utils import  solver_iter_krylov
from skfem.models.general import divergence
from skfem.models.poisson import vector_laplace
from skfem.helpers import grad, dot, laplacian
from skfem.helpers import laplacian, precompute_operators
import matplotlib.pyplot as plt

import scipy.sparse as sp
from scipy.sparse.linalg import eigsh, eigs, splu, LinearOperator



import plotly.graph_objects as go
from skfem.mesh import MeshTet # MeshTet para mallas tetraédricas (3D)

In [8]:
# Mesh
p    = np.linspace(0, 1, 10)
mesh = MeshTet.init_tensor(*(p,) * 3)

# Asignar ID a las fronteras
mesh = mesh.with_boundaries({
    'dirichlet' :  lambda x: (x[0] == 0.0) | (x[0] == 1.0) | 
                             (x[1] == 0.0) | (x[1] == 1.0) | 
                             (x[2] == 0.0) | (x[2] == 1.0),
})

In [51]:
# Definir elementos y bases (P2 para velocidad, P1 para presión) en 3D
element = {
    'u': ElementVector(ElementTetP2(), dim=3),
    'p': ElementTetP1(),
}
basis = {
    'u': Basis(mesh, element['u'], intorder=4),
    'p': Basis(mesh, element['p'], intorder=4),
}
basis_u, basis_p = basis['u'], basis['p']
Nu, Np = basis_u.N, basis_p.N
N      = Nu + Np

In [52]:
@BilinearForm
def mass_matrix(u, v, w):
    return dot(u, v)

# Término convectivo linealizado
@BilinearForm
def convection(u, v, w):
    advection_field = w['w']
    grad_u = grad(u)
    return np.einsum('j...,ij...,i...->...', advection_field, grad_u, v)


# Ensamblaje de matrices
A =  asm(vector_laplace, basis_u)               
B = -asm(divergence, basis_u, basis_p)   

In [53]:
def u_exact_x(x, y, z):
    return y**2
def u_exact_y(x, y, z):
    return -(z**2)
def u_exact_z(x, y, z):
    return x**2
def p_exact(x, y, z):
    return 0*x + 0*y + 0*z + 2
def f_exact_x(x, y, z):
    return -2*y*z**2 - 2
def f_exact_y(x, y, z):
    return -2*z*x**2 + 2
def f_exact_z(x, y, z):
    return 2*x*y**2 - 2

In [55]:
x_boundary = np.zeros(Nu + Np)

# DOFs de las fronteras
dofs_dirichlet  = basis_u.get_dofs().all()
dofs_dirichlet_p  = basis_p.get_dofs().all()

x_bd = basis_u.doflocs[0, dofs_dirichlet[::3]]
y_bd = basis_u.doflocs[1, dofs_dirichlet[1::3]]
z_bd = basis_u.doflocs[2, dofs_dirichlet[2::3]]

x_boundary[dofs_dirichlet[::3]]  = u_exact_x(x_bd, y_bd, z_bd)
x_boundary[dofs_dirichlet[1::3]] = u_exact_y(x_bd, y_bd, z_bd)
x_boundary[dofs_dirichlet[2::3]] = u_exact_z(x_bd, y_bd, z_bd)
x_boundary[Nu + dofs_dirichlet_p[0]] = p_exact(basis_p.doflocs[0, dofs_dirichlet_p[0]],
                                               basis_p.doflocs[1, dofs_dirichlet_p[1]],    
                                               basis_p.doflocs[2, dofs_dirichlet_p[2]])

dofs_boundary = np.concatenate([
    dofs_dirichlet,
    np.array([Nu + dofs_dirichlet_p[0]])
])

In [57]:
@LinearForm
def rhs_u(v, w):
    x, y, z= w.x
    fx = f_exact_x(x, y, z)
    fy = f_exact_y(x, y, z)
    fz = f_exact_z(x, y, z)
    return fx * v[0] + fy * v[1] + fz * v[2]
b_u = asm(rhs_u, basis_u) 
b_p = np.zeros(Np)  
F   = np.concatenate([b_u, b_p])

In [58]:
# Término convectivo linealizado
@BilinearForm
def convection(u, v, w):
    advection_field = w['w']
    grad_u = grad(u)
    return np.einsum('j...,ij...,i...->...', advection_field, grad_u, v)

@BilinearForm
def convection2(u, v, w):
    advection_field = u
    grad_w = grad(w['w'])
    return np.einsum('j...,ij...,i...->...', advection_field, grad_w, v)



In [40]:
def solve_ns_newton(u_init, p_init, Re, max_iter, tol):
    u = u_init
    p = p_init
    for it in range(max_iter):
        # Campo de advección congelado w := u^(it) en puntos de cuadratura
        W = basis_u.interpolate(u)   

        # Ensambla bloque convectivo C(w)
        C2 = asm(convection, basis_u, w=W)

        # Ensambla derivada del bloque convectivo C'(w)
        C1 = asm(convection2, basis_u, w=W)

        # Matriz bloque del paso linealizado
        DF = bmat([[(1/Re) * A + C1 + C2, B.T ],
                  [B,                        None]], format='csr')

        F1 = C2*u + (1/Re)*A*u + B.T*p 
        F2 = B*u
        F_real = np.concatenate([F1, F2])

        # Resolver
        delta = solve(*condense(DF, F_real, D=dofs_boundary, x=x_boundary*0))
        u_new = u - delta[:Nu]
        p_new = p - delta[Nu:Nu+Np]

        # Criterio de convergencia
        du = u_new - u
        rel_u = np.linalg.norm(du) / (np.linalg.norm(u_new) + 1e-16)

        dp = p_new - p
        rel_p = np.linalg.norm(dp) / (np.linalg.norm(p_new) + 1e-16)

        # # Sub-relajación si se desea
        u = u_new
        p = p_new

        # print(f"Iteración {it+1}: rel_u = {rel_u:.4e}, rel_p = {rel_p:.4e}")

        if rel_u < tol and rel_p < tol:
            print(f"Convergió en {it+1} iteraciones, residuo {max(rel_u, rel_p):.4e}")
            return u, p, True
    print("No convergió en el número máximo de iteraciones")
    return u, p, False

In [59]:
def solve_ns_picard(u_init, p_init, Re, max_iter, tol):
    u = u_init
    p = p_init
    for it in range(max_iter):
        # Campo de advección congelado w := u^(it) en puntos de cuadratura
        W = basis_u.interpolate(u)   

        # Ensambla bloque convectivo C(w)
        C = asm(convection, basis_u, w=W)

        # Matriz bloque del paso linealizado
        K = bmat([[(1/Re) * A + C, B.T ],
                  [B,              None]], format='csr')

        # Resolver
        sol = solve(*condense(K, F, D=dofs_boundary, x=x_boundary))
        u_new = sol[:Nu]
        p_new = sol[Nu:Nu+Np]

        # Criterio de convergencia
        du = u_new - u
        rel_u = np.linalg.norm(du) / (np.linalg.norm(u_new) + 1e-16)

        dp = p_new - p
        rel_p = np.linalg.norm(dp) / (np.linalg.norm(p_new) + 1e-16)

        # # Sub-relajación si se desea
        u = u_new
        p = p_new

        if rel_u < tol and rel_p < tol:
            print(f"Convergió en {it+1} iteraciones, residuo {max(rel_u, rel_p):.4e}")
            return u, p, True
    print("No convergió en el número máximo de iteraciones")
    return u, p, False

In [63]:
mu = 1.0

# Calcular solución inicial de stokes
K_stokes = bmat([[mu * A,     B.T],  
                 [B,         None]], format='csr')

sol0 = solve(*condense(K_stokes, F, D=dofs_boundary, x=x_boundary))
u_ref = sol0[:Nu].copy()
p_ref = sol0[Nu:Nu+Np].copy()

In [64]:
# Resolver incrementanto Re
Re = 100
Re_linspace = np.linspace(100, Re, 2)

for R in Re_linspace:
    print(f"Resolviendo para Re = {R:.2f}")
    u_ref, p_ref, flag = solve_ns_picard(u_ref, p_ref, R, max_iter=500, tol=1e-12)

    if not flag:
        print(f"No se pudo converger para este Re = {R:.2f}.")
        break

u_sol = u_ref
p_sol = p_ref


Resolviendo para Re = 100.00
Convergió en 16 iteraciones, residuo 7.4914e-13
Resolviendo para Re = 100.00
Convergió en 1 iteraciones, residuo 2.7309e-13


In [66]:
P1_basis = Basis(mesh, ElementTetP1())

# Mapeo de Presión (Escalar)
pressure_scalar = p_sol 

# Mapeo de Velocidad (Vectorial 3D)
velocity_vector = np.zeros((mesh.nvertices, 3))

for idx in range(mesh.nvertices):
    velocity_vector[idx, 0] = u_sol[basis_u.nodal_dofs[0, idx]]  # componente X
    velocity_vector[idx, 1] = u_sol[basis_u.nodal_dofs[1, idx]]  # componente Y
    velocity_vector[idx, 2] = u_sol[basis_u.nodal_dofs[2, idx]]  # componente Z

point_data_to_export = {
    "velocity": velocity_vector,
    "pressure": pressure_scalar,
}

mesh.save(
        "navierstokes_example3D.vtu", 
        point_data=point_data_to_export,
)

### Strong error

In [32]:
### Error forma fuerte
basis_x = basis_u.split_bases()[0]
basis_y = basis_u.split_bases()[1]
basis_z = basis_u.split_bases()[2]
u_x = u_ref[0::3]
u_y = u_ref[1::3]
u_z = u_ref[2::3] 
p_sol = p_ref 

edofs_x, phix, grad_phix, laplacian_phix = precompute_operators(basis_x)
edofs_y, phiy, grad_phiy, laplacian_phiy = precompute_operators(basis_y)
edofs_z, phiz, grad_phiz, laplacian_phiz = precompute_operators(basis_z)
edofs_p, phip, grad_phip, laplacian_phip = precompute_operators(basis_p)

In [33]:
# Velocidad y presión en puntos de cuadratura
u_val = np.einsum('dq, de -> qe', phix, u_x[edofs_x]) 
v_val = np.einsum('dq, de -> qe', phiy, u_y[edofs_y])
w_val = np.einsum('dq, de -> qe', phiz, u_z[edofs_z])
p_val = np.einsum('dq, de -> qe', phip, p_sol[edofs_p])

# Calcular operadores en coordenadas físicas
grad_u_fisico_x = np.einsum('de, diqe -> iqe', u_x[edofs_x]  , grad_phix)
grad_u_fisico_y = np.einsum('de, diqe -> iqe', u_y[edofs_y]  , grad_phiy)
grad_u_fisico_z = np.einsum('de, diqe -> iqe', u_z[edofs_z]  , grad_phiz)
grad_p_fisico   = np.einsum('de, diqe -> iqe', p_sol[edofs_p], grad_phip)
laplacian_u_x   = np.einsum('de, de  -> e'  , u_x[edofs_x]  , laplacian_phix)
laplacian_u_y   = np.einsum('de, de   -> e'  , u_y[edofs_y]  , laplacian_phiy)
laplacian_u_z   = np.einsum('de, de   -> e'  , u_z[edofs_z]  , laplacian_phiz)

In [35]:
# Residuo momentum en X:

res_momentum_x = np.abs(grad_p_fisico[0] - (laplacian_u_x)/Re + (u_val * grad_u_fisico_x[0] + v_val * grad_u_fisico_x[1] + w_val * grad_u_fisico_x[2]) )

# Residuo momentum en Y:
res_momentum_y = np.abs(grad_p_fisico[1] - (laplacian_u_y)/Re + (u_val * grad_u_fisico_y[0] + v_val * grad_u_fisico_y[1] + w_val * grad_u_fisico_y[2]) )

# Residuo momentum en Z: 
res_momentum_z = np.abs(grad_p_fisico[2] - (laplacian_u_z)/Re + (u_val * grad_u_fisico_z[0] + v_val * grad_u_fisico_z[1] + w_val * grad_u_fisico_z[2]) )

# Residuo Continuidad:
res_continuity = np.abs(grad_u_fisico_x[0] + grad_u_fisico_y[1] + grad_u_fisico_z[2])

print(f"Residuo momentum en X - min: {np.min(res_momentum_x):.4e}, max: {np.max(res_momentum_x):.4e}")
print(f"Residuo momentum en Y - min: {np.min(res_momentum_y):.4e}, max: {np.max(res_momentum_y):.4e}")
print(f"Residuo momentum en Z - min: {np.min(res_momentum_z):.4e}, max: {np.max(res_momentum_z):.4e}")
print(f"Residuo continuidad - min: {np.min(res_continuity):.4e}, max: {np.max(res_continuity):.4e}")


Residuo momentum en X - min: 3.9398e-07, max: 7.2344e+00
Residuo momentum en Y - min: 2.5933e-06, max: 8.3132e+00
Residuo momentum en Z - min: 6.2850e-07, max: 1.1635e+01
Residuo continuidad - min: 8.0110e-07, max: 4.1779e+00


In [22]:
# Calcular el máximo error por elemento para cada tipo de residuo
max_res_momentum_x_per_elem = np.max(res_momentum_x, axis=0)
max_res_momentum_y_per_elem = np.max(res_momentum_y, axis=0)
max_res_momentum_z_per_elem = np.max(res_momentum_z, axis=0)
max_res_continuity_per_elem = np.max(res_continuity, axis=0)

# Top 10 elementos con mayor error en momentum X
top_10_idx_momentum_x = np.argsort(max_res_momentum_x_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Momentum X:")
for i, idx in enumerate(top_10_idx_momentum_x):
    print(f"{i+1}. Elemento {idx}: {max_res_momentum_x_per_elem[idx]:.6e}")

print("\n" + "="*60 + "\n")

# Top 10 elementos con mayor error en momentum Y
top_10_idx_momentum_y = np.argsort(max_res_momentum_y_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Momentum Y:")
for i, idx in enumerate(top_10_idx_momentum_y):
    print(f"{i+1}. Elemento {idx}: {max_res_momentum_y_per_elem[idx]:.6e}")

print("\n" + "="*60 + "\n")

# Top 10 elementos con mayor error en momentum Y
top_10_idx_momentum_z = np.argsort(max_res_momentum_z_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Momentum Z:")
for i, idx in enumerate(top_10_idx_momentum_z):
    print(f"{i+1}. Elemento {idx}: {max_res_momentum_z_per_elem[idx]:.6e}")

print("\n" + "="*60 + "\n")

# Top 10 elementos con mayor error en continuidad
top_10_idx_continuity = np.argsort(max_res_continuity_per_elem)[-100:][::-1]
print("Top 10 elementos con mayor residuo en Continuidad:")
for i, idx in enumerate(top_10_idx_continuity):
    print(f"{i+1}. Elemento {idx}: {max_res_continuity_per_elem[idx]:.6e}")

Top 10 elementos con mayor residuo en Momentum X:
1. Elemento 4360: 7.234430e+00
2. Elemento 2902: 7.006384e+00
3. Elemento 4361: 6.955632e+00
4. Elemento 2903: 6.623555e+00
5. Elemento 3600: 6.420948e+00
6. Elemento 4329: 6.341242e+00
7. Elemento 4351: 6.241607e+00
8. Elemento 4338: 6.226470e+00
9. Elemento 2901: 6.217930e+00
10. Elemento 3609: 6.206248e+00
11. Elemento 4359: 6.190460e+00
12. Elemento 4352: 6.160479e+00
13. Elemento 2893: 6.058853e+00
14. Elemento 4362: 5.919096e+00
15. Elemento 2894: 5.880949e+00
16. Elemento 3591: 5.670872e+00
17. Elemento 3632: 5.617707e+00
18. Elemento 3631: 5.602237e+00
19. Elemento 2880: 5.585386e+00
20. Elemento 1434: 5.570607e+00
21. Elemento 4350: 5.566552e+00
22. Elemento 4353: 5.564741e+00
23. Elemento 2892: 5.518693e+00
24. Elemento 4320: 5.348912e+00
25. Elemento 2163: 5.332029e+00
26. Elemento 4347: 5.327999e+00
27. Elemento 1435: 5.264891e+00
28. Elemento 3618: 5.214572e+00
29. Elemento 2904: 5.214521e+00
30. Elemento 2889: 5.155009e+00

In [23]:
# Índices de DOFs a analizar
index_dof = top_10_idx_momentum_z[:7]

# Obtener elementos y tetraedros
elements = mesh.t.T
nodes_coords = mesh.p

# Crear figura
fig = go.Figure()

# 1. Graficar la malla completa (semitransparente)
# Extraer todas las caras del boundary
boundary_facets_idx = mesh.boundary_facets()
boundary_faces = mesh.facets[:, boundary_facets_idx]

fig.add_trace(
    go.Mesh3d(
        x=nodes_coords[0, :],
        y=nodes_coords[1, :],
        z=nodes_coords[2, :],
        i=boundary_faces[0, :],
        j=boundary_faces[1, :],
        k=boundary_faces[2, :],
        color='lightgray',
        opacity=0.2,
        name='Malla completa',
        showlegend=True,
        lighting=dict(ambient=0.6, diffuse=0.5, specular=0.3)
    )
)

# 2. Para cada DOF, encontrar elementos que lo contienen
colors = ['red', 'blue', 'green', 'orange', 'purple', 'cyan', 'magenta']

for idx, dof in enumerate(index_dof):
    # Encontrar elementos que contienen este DOF
    element_indices = np.where(basis_u.element_dofs == dof)[1]
    
    if len(element_indices) == 0:
        continue
    
    # Obtener tetraedros de esos elementos
    elements_subset = elements[element_indices]
    
    # Crear caras de los tetraedros para visualización
    # Cada tetraedro tiene 4 caras triangulares
    faces_i, faces_j, faces_k = [], [], []
    
    for tet in elements_subset:
        # Cara 1: nodos 0, 1, 2
        faces_i.extend([tet[0], tet[0], tet[0], tet[1]])
        faces_j.extend([tet[1], tet[1], tet[2], tet[2]])
        faces_k.extend([tet[2], tet[3], tet[3], tet[3]])
    
    # Graficar elementos que contienen el DOF
    fig.add_trace(
        go.Mesh3d(
            x=nodes_coords[0, :],
            y=nodes_coords[1, :],
            z=nodes_coords[2, :],
            i=faces_i,
            j=faces_j,
            k=faces_k,
            color=colors[idx % len(colors)],
            opacity=0.6,
            name=f'Soporte DOF {dof}',
            showlegend=True
        )
    )
    
    # 3. Marcar la ubicación exacta del DOF
    dof_loc = basis_u.doflocs[:, dof]
    
    fig.add_trace(
        go.Scatter3d(
            x=[dof_loc[0]],
            y=[dof_loc[1]],
            z=[dof_loc[2]],
            mode='markers',
            marker=dict(
                size=8,
                color=colors[idx % len(colors)],
                symbol='diamond',
                line=dict(color='black', width=2)
            ),
            name=f'DOF {dof}',
            showlegend=True
        )
    )

# Configurar layout con mayor ancho
fig.update_layout(
    title='Soporte local de DOFs en Malla 3D',
    width=800,
    height=600,
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()